# Tutorial 4
## Name: Soham Sahasrabuddhe
## Roll Number: 23B1848

In [1]:
import numpy as np
import pandas as pd
from ucimlrepo import fetch_ucirepo 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

### Question 1

In [2]:
# Defining activation function
def step(x):
    return np.where(x > 0, 1, 0)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(x):
    return np.maximum(x, 0)

def tanh(x):
    return np.tanh(x)

# Dictionary for input of activation
activation_functions = {
    'step': step,
    'sigmoid': sigmoid,
    'relu': relu,
    'tanh': tanh
    }

In [12]:
# User inputs on size, weights, biases and activation
n_i = int(input("Neurons in the input layer: "))
n_h = int(input("Neurons in the hidden layer: "))
n_o = int(input("Neurons in the output layer: "))

m_h = np.array([int(input(f'Number of neurons in the hidden layer {i+1}: ')) for i in range(n_h)])

inputs = np.array([float(input(f'Enter real input value {i+1}: ')) for i in range(n_i)])

weights = []
biases = []

# input to hidden layer
weights.append(np.array([[float(input(f"Enter weight for Input layer to first hidden layer i.e. W(1)_{i + 1}{j + 1}: ")) for j in range(m_h[0])] for i in range(n_i)]))
biases.append(np.array([float(input(f"Enter bias for Input layer to first hidden layer i.e. b(1)_{i + 1}: ")) for i in range(m_h[0])]))

# hidden layers
for k in range(1,n_h):
    weights.append(np.array([[float(input(f"Enter weight for Hidden layer {k} to Hidden Layer {k + 1} i.e. W({k + 1})_{i + 1}{j + 1}: ")) for j in range(m_h[k])] for i in range(m_h[k - 1])]))
    biases.append(np.array([float(input(f"Enter bias for Hidden layer {k} to Hidden Layer {k + 1} i.e. b({k + 1})_{i +1}: ")) for i in range(m_h[k])]))

# hidden to output layer
weights.append(np.array([[float(input(f"Enter weight for Hidden layer {n_h} to Output layer i.e. W({n_h + 1})_{i + 1}{j +1}: ")) for j in range(n_o)] for i in range(m_h[n_h- 1])]))
biases.append(np.array([float(input(f"Enter bias for Hidden layer {n_h} to Output layer i.e. b({n_h + 1})_{i + 1}: ")) for i in range(n_o)]))

# Activation
activation_user = input("Choose appropriate function out of step, relu, sigmoid, and tanh: ").lower()  # For consistency if user types uppercase characters
activation = activation_functions.get(activation_user)
if activation is None:
    print("Invalid activation function. Defaulting to sigmoid.")
    activation = sigmoid
else:
    print(f"Activation function chosen: {activation_user}")

Activation function chosen: step


In [13]:
# Forward Propagation
input_layer = inputs

for k in range(n_h + 1):
    output_layer = np.dot(input_layer, weights[k]) + biases[k]
    input_layer = activation(output_layer) if k < n_h else output_layer

final_output = input_layer
print("Output: ", final_output)

Output:  [3.]


### Question 2:

In [6]:
# Parameters and defining the dataset
N = 20

x_rnd = np.random.rand(N*N)
X_1 = np.linspace(-0.5,0.5,N)
X_2 = np.linspace(-0.5,0.5,N)
x1,x2 = np.meshgrid(X_1,X_2)
x1 = x1.flatten()
x2 = x2.flatten()
X = np.vstack((x1, x2)).T

y = x1**2 + 2*x1 - x2 + x2**2 + 0.5*x_rnd

# Division into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=300, test_size=100, random_state=37)

In [3]:
def training_network(X_train, y_train, X_test, y_test, layers, neurons, atv, epochs, batchsize):
    # Define the model
    model = Sequential()

    model.add(Dense(neurons, activation = atv)) # Input

    for i in range(layers):   #Hidden
        model.add(Dense(neurons, activation = atv))

    model.add(Dense(1)) # Output
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    model.fit(X_train, y_train, epochs = epochs, batch_size = batchsize, verbose=0) # Training

    loss = model.evaluate(X_test, y_test, verbose = 0)
    return loss

In [24]:
hidden_confg = [1, 2, 3]
neuron_confg = [10, 20 ,30]
activation =  ['relu', 'tanh', 'sigmoid']
epochs = [10, 20, 50, 100]
batch_size = [32, 64, 128]
results = []

for layer in hidden_confg:
    for neuron in neuron_confg:
        for act in activation:
            for ep in epochs:
                for bs in batch_size:
                    test_loss = training_network(X_train, y_train, X_test, y_test, layers = layer, neurons=neuron, atv=act, epochs=ep, batchsize=bs)
                    results.append([layer, neuron, act,ep,bs, test_loss])

results.append({
    'Hidden Layers': layer,
    'Neurons': neuron,
    'Activation': act,
    'Epochs': ep,
    'Batch Size': bs,
    'Test Loss': test_loss
})

results = pd.DataFrame(results)
results.columns = ['Hidden Layers', 'Neurons', 'Activation', 'Epochs','Batch Size', 'Test Loss']
print(results)

     Hidden Layers  Neurons  Activation  Epochs  Batch Size  Test Loss
0                1       10        relu      10          32   0.148151
1                1       10        relu      10          64   0.390454
2                1       10        relu      10         128   0.561285
3                1       10        relu      20          32    0.03409
4                1       10        relu      20          64   0.180774
..             ...      ...         ...     ...         ...        ...
320              3       30     sigmoid      50         128   0.365373
321              3       30     sigmoid     100          32   0.036598
322              3       30     sigmoid     100          64    0.03326
323              3       30     sigmoid     100         128   0.041554
324  Hidden Layers  Neurons  Activation  Epochs  Batch Size  Test Loss

[325 rows x 6 columns]


### Question 3

In [4]:
# fetch dataset 
liver_disorders = fetch_ucirepo(id=60) 
  
# data (as pandas dataframes) 
X = np.array(liver_disorders.data.features)
y = np.array(liver_disorders.data.targets)
  
# metadata 
print(liver_disorders.metadata) 
  
# variable information 
print(liver_disorders.variables)

{'uci_id': 60, 'name': 'Liver Disorders', 'repository_url': 'https://archive.ics.uci.edu/dataset/60/liver+disorders', 'data_url': 'https://archive.ics.uci.edu/static/public/60/data.csv', 'abstract': 'BUPA Medical Research Ltd. database donated by Richard S. Forsyth', 'area': 'Health and Medicine', 'tasks': ['Regression'], 'characteristics': ['Multivariate'], 'num_instances': 345, 'num_features': 5, 'feature_types': ['Categorical', 'Integer', 'Real'], 'demographics': [], 'target_col': ['drinks'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2016, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C54G67', 'creators': [], 'intro_paper': None, 'additional_info': {'summary': 'The first 5 variables are all blood tests which are thought to be sensitive to liver disorders that might arise from excessive alcohol consumption. Each line in the dataset constitutes the record of a single male individual.\n\nImportant note: The 7

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=273, test_size=72, random_state=42)

In [6]:
hidden_confg = [1, 2, 3]
neuron_confg = [10, 20 ,30]
activation =  ['relu', 'tanh', 'sigmoid']
results = []

for layer in hidden_confg:
    for neuron in neuron_confg:
        for act in activation:
            test_loss = training_network(X_train, y_train, X_test, y_test, layers = layer, neurons=neuron, atv=act, epochs=100, batchsize=64)
            results.append([layer, neuron, act, test_loss])

results.append({
    'Hidden Layers': layer,
    'Neurons': neuron,
    'Activation': act,
    'Test Loss': test_loss
})

results = pd.DataFrame(results)
print(results)

                0        1           2          3
0               1       10        relu  25.832396
1               1       10        tanh   9.816648
2               1       10     sigmoid  10.086171
3               1       20        relu  11.204685
4               1       20        tanh   8.832933
5               1       20     sigmoid   9.691667
6               1       30        relu  11.102077
7               1       30        tanh   8.739369
8               1       30     sigmoid   9.024007
9               2       10        relu  10.659854
10              2       10        tanh  10.452765
11              2       10     sigmoid  10.662361
12              2       20        relu  13.061833
13              2       20        tanh   9.606407
14              2       20     sigmoid   10.06094
15              2       30        relu  10.346254
16              2       30        tanh     9.6021
17              2       30     sigmoid   9.019244
18              3       10        relu    9.99665
